In [3]:
import os
import shutil

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pandas as pd
import numpy as np
import math

import re

In [9]:
# Process Pokémon and one-hot encode the types

pokemon = pd.read_csv('../data/external/stats/pokemon.csv')
pokemon["Variety"] = float("NaN")
pokemon = pokemon[["#", "Name", "Variety", "Type 1", "Type 2"]]

# Create one-hot columns for each type
types = set(pokemon["Type 1"])
for t in types:
    pokemon["is" + str(t)] = 0

# Iterate over Pokémon
for i, p in pokemon.iterrows():
    
    # Clean up name
    pokemonName = pokemon.loc[i, "Name"]
    pokemonName = pokemonName.replace(" ", "")
    
    pokemonNameComponents = re.findall('[A-Z][^A-Z0-9]*', pokemonName)

    pokemon.loc[i, "Name"] = pokemonNameComponents[0]
    
    # Set variety (Mega etc.)
    if len(pokemonNameComponents) > 1:
        pokemonNameComponents = [value for value in pokemonNameComponents if value != pokemonNameComponents[0]]
        pokemon.loc[i, "Variety"] = str(pokemonNameComponents).lower()
    else:
        pokemon.loc[i, "Variety"] = "normal"
    
    #  Set one-hot columns to 1 for relevant types
    pokemon.loc[i, "is" + p["Type 1"]] = 1

    if not pd.isna(p["Type 2"]):
        pokemon.loc[i, "is" + p["Type 2"]] = 1

# Ditch all Pokemon with varieties (e.g. Megas):
pokemon = pokemon[pokemon["Variety"] == "normal"]


# Save output
pokemon.to_csv('../data/processed/pokemon.csv', index=False)

pokemon.head()

   #        Name Variety Type 1  Type 2  isPoison  isFlying  isNormal  \
0  1   Bulbasaur  normal  Grass  Poison         1         0         0   
1  2     Ivysaur  normal  Grass  Poison         1         0         0   
2  3    Venusaur  normal  Grass  Poison         1         0         0   
4  4  Charmander  normal   Fire     NaN         0         0         0   
5  5  Charmeleon  normal   Fire     NaN         0         0         0   

   isElectric  isDragon  ...  isFighting  isSteel  isIce  isGhost  isFairy  \
0           0         0  ...           0        0      0        0        0   
1           0         0  ...           0        0      0        0        0   
2           0         0  ...           0        0      0        0        0   
4           0         0  ...           0        0      0        0        0   
5           0         0  ...           0        0      0        0        0   

   isWater  isPsychic  isDark  isGround  isRock  
0        0          0       0         0   

In [13]:
# Process image data

data_directory_images = "../data/external/images-gen-1-6"
output_directory = "../data/processed/pokemon"

pokemon = pd.read_csv('../data/processed/pokemon.csv')
pokemon["imagePath"] = np.nan

# Remove processed folder and create empty new one
try:
    shutil.rmtree(output_directory)
    os.mkdir(output_directory)
except:
    os.mkdir(output_directory)

# Copy images to processed folder
for image in os.listdir(data_directory_images):
    pokemon_id = image.split('.')[0]

    # Images with no variety (e.g. "211.png")
    if pokemon_id.isnumeric():
        
        # Copy to processed folder
        src = data_directory_images + "/" + image
        dst = output_directory + "/" + pokemon_id + ".png"

        shutil.copyfile(src, dst)

        # Set image path in data frame
        pokemon.loc[pokemon["#"] == int(pokemon_id), 'imagePath'] = dst
        

# Drop Pokemon without image path
pokemon.dropna()

# Save pokemon.csv with image paths
pokemon.to_csv('../data/processed/pokemon-with-image-paths.csv', index=False)

pokemon.head()

,#,Name,Variety,Type 1,Type 2,isPoison,isFlying,isNormal,isElectric,isDragon,...,isSteel,isIce,isGhost,isFairy,isWater,isPsychic,isDark,isGround,isRock,imagePath
0,1,Bulbasaur,normal,Grass,Poison,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,../data/processed/pokemon/1.png
1,2,Ivysaur,normal,Grass,Poison,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,../data/processed/pokemon/2.png
2,3,Venusaur,normal,Grass,Poison,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,../data/processed/pokemon/3.png
3,4,Charmander,normal,Fire,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,../data/processed/pokemon/4.png
4,5,Charmeleon,normal,Fire,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,../data/processed/pokemon/5.png
